# Ödev 1

## Soru 1

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({
    "alinan_urun": ['kazak', 'mont', 'kazak', 'pantolon', 'mont'],
    "memnuniyet": ['cok iyi', 'iyi', 'cok iyi', 'orta', 'iyi'],
    "satici_yildizi(1-5)": [4,3,5,1,4],
    "fiyat": [50,150,70,100,300]
})

df.columns = pd.MultiIndex.from_tuples([('nominal','alinan_urun'), ('ordinal','memnuniyet'), ('interval', 'satici_yildizi(1-5)'), ('ratio', 'fiyat')], names=['Level of Measurement','Column Name'])
df

Level of Measurement,nominal,ordinal,interval,ratio
Column Name,alinan_urun,memnuniyet,satici_yildizi(1-5),fiyat
0,kazak,cok iyi,4,50
1,mont,iyi,3,150
2,kazak,cok iyi,5,70
3,pantolon,orta,1,100
4,mont,iyi,4,300


## Soru 2

In [3]:
columns = ['radius_mean', 'area_mean', 'compactness_mean']
data = pd.read_csv('data.csv', usecols=columns)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   radius_mean       569 non-null    float64
 1   area_mean         569 non-null    float64
 2   compactness_mean  569 non-null    float64
dtypes: float64(3)
memory usage: 13.5 KB


In [4]:
description = data.describe().T #T just for better look
def outlier_count(x, q1, q3): #Takes a series and quartiles of that series, returns outlier count.
    iqr = q3 - q1
    return sum(~x.between(q1-iqr * 1.5, q3 + iqr*1.5))
    
description['outlier_count'] = description.apply(lambda x: outlier_count(data[x.name], x.loc['25%'], x.loc['75%']), axis=1)
description

,count,mean,std,min,25%,50%,75%,max,outlier_count
radius_mean,569.0,14.127292,3.524049,6.98100,11.70000,13.37000,15.7800,28.1100,14
area_mean,569.0,654.889104,351.914129,143.50000,420.30000,551.10000,782.7000,2501.0000,25
compactness_mean,569.0,0.104341,0.052813,0.01938,0.06492,0.09263,0.1304,0.3454,16


#### Yorumlar:
- Count sayılarından anlaşıldığı üzere eksik veri yok, 3 sütunda da veriler tam.
- Mean değeri ve median değeri radius ve compactness için çok yakın, burdan dağılımın daha düzenli olduğunu ve normal dağılıma yakın olma ihtimali olduğu çıkarılabilir. Area'da ise mean ve median değerleri arasındaki farkın yaklaşık %20 civarında olduğu görülmektedir, bu farktan dolayı bu sütunla ilgili işlem yaparken dikkatli olunması gerekebilir.
- Standart sapma değerleri ise area ve compactness sütunlarında ortalamanın yarısı kadardır, radius'ta ise ortalamasına göre çok daha düşüktür. Buradan radius'un ortalama etrafında yakın değerlere dağıldığı fakat area ve compactness'ın daha yaygın olduğu anlaşılabilir. Eğer normal dağılmışlarsa radius'un daha sivri bir grafiğe sahip olması beklenir, diğer ikisinin ise daha basık ve yaygın bir grafiğe sahip olması beklenir.
- Quartile değerlerine bakacak olursak radius ve compactness'ta median ile lower quartile arasındaki fark ve median ile upper quartile arasındaki fark yaklaşık eşittir, buradan herhangi bir yana yatık olmama ihtimalinin yüksek olduğunu düşünebiliriz. Area sütununda ise lower quartile, upper quartile'a göre median'a çok daha yakındır, buradan area'nın dağılımıyla ilgili bir şüphe daha oluşabilir.
- Outlier sayıları radius ve compactness'da toplam verinin yaklaşık %3'ü kadardır, area'da ise yaklaşık %5'i kadardır. Area'da range'in de ortalmasına oranla büyük olmasından dolayı outlier sayısının daha fazla olması da beklenen bir sonuçtu.
- Son olarak area'nın ortalamasının yaklaşık olarak radius'un ortalmasının karesinin pi ile çarpımı kadar olması beklenir(dairenin alanından dolayı). Beklendiği gibi yaklaşık öyle çıkmıştır.

## Soru 3

### 3.1.Correlation

In [5]:
data.corr() #correlation

,radius_mean,area_mean,compactness_mean
radius_mean,1.000000,0.987357,0.506124
area_mean,0.987357,1.000000,0.498502
compactness_mean,0.506124,0.498502,1.000000


### 3.2.Covariance

In [6]:
data.cov() #covariance

,radius_mean,area_mean,compactness_mean
radius_mean,12.418920,1224.483409,0.094197
area_mean,1224.483409,123843.554318,9.264931
compactness_mean,0.094197,9.264931,0.002789


### 3.3.Pearson Correlation Coefficient

In [7]:
data.corr(method='pearson') # DataFrame.corr() method uses pearson method defaultly

,radius_mean,area_mean,compactness_mean
radius_mean,1.000000,0.987357,0.506124
area_mean,0.987357,1.000000,0.498502
compactness_mean,0.506124,0.498502,1.000000


### 3.4.Spearman Rank Coefficient

In [8]:
data.corr(method='spearman')

,radius_mean,area_mean,compactness_mean
radius_mean,1.000000,0.999602,0.497578
area_mean,0.999602,1.000000,0.488988
compactness_mean,0.497578,0.488988,1.000000


### 3.5. Effect Size
Effect size veya daha spesifik olarak cohen's d formulü şöyledir:
$$ d = \frac{\mu_1 - \mu_2}{s}\ ; \qquad s ={\sqrt {\frac {(n_{1}-1)s_{1}^{2}+(n_{2}-1)s_{2}^{2}}{n_{1}+n_{2}-2}}} $$
Bizim verilerimizde tüm verilerin count'u eşit olduğu için $n_1 = n_2$'dir.
$$ n_1 = n_2 = n\quad \Rightarrow \quad s ={\sqrt {\frac {(n-1)s_{1}^{2}+(n-1)s_{2}^{2}}{n+n-2}}} = {\sqrt {\frac {(n-1)(s_{1}^{2}+s_{2}^{2})}{2(n-1)}}} = {\sqrt {\frac {(s_{1}^{2}+s_{2}^{2})}{2}}} $$

Formülün bu sadeleşmiş halini kullanabiliriz:
$$ s = \sqrt{\frac{s_1^2 + s_2^2}{2}} $$

In [9]:
def cohen_d(x1,x2):
    s = np.sqrt((np.std(x1)**2 + np.std(x2)**2 )/ 2)
    return (np.mean(x1) - np.mean(x2))/s

In [10]:
effect_sizes = {}
for i in data.columns:
    l = []
    for j in data.columns:
        l.append(cohen_d(data[i],data[j]))
    effect_sizes[i] = l
pd.DataFrame(effect_sizes, index=data.columns)

,radius_mean,area_mean,compactness_mean
radius_mean,0.000000,2.577123,-5.631780
area_mean,-2.577123,0.000000,-2.633655
compactness_mean,5.631780,2.633655,0.000000


## Yorumlar:
- Radius ile area arasındaki korelasyon 1'e çok yakın bir değer çıktı. Bu da bu iki değişkenin neredeyse tamamen korele olduğu anlamına gelmektedir. area = $\pi r^2$ olduğu için ve $\pi$ sabit bir sayı olduğu için bu iki değerin tamamen korele olması beklenen bir sonuçtu. Bu iki sütun tamamen birbirinin tekrarıdır.
- Compactness sütununun ise diğer iki sütunla korelasyonu yaklaşık 0.5 çıkmıştır. Bu değer ise compactness değişkeninin radius ve area ile birlikte değiştiğini fakat ilk örnekteki gibi tamamen birbirinin tekrarı olmadığını göstermektedir. Ayrıca işaretinin pozitif olması da biri artarken diğerinin de arttığını göstermektedir.
- Pearson correlation coefficient'in sonucu ilk baştaki korelasyonla tamamen aynı çıkmıştır çünkü pandas kütüphanesindeki corr() metodu default olarak pearson korelasyonunu kullanarak hesaplamaktadır.
- Spearman Rank Coefficient değerleri ise pearson'daki sonuçlara çok yakın çıkmıştır. Spearman monotonik ilişkinin bir göstergesi olduğu için ve pearson lineer ilişkinin göstergesi olduğu için ve alan ve yarıçap arasındaki ilişki lineer olmadığı için spearman korelasyonu daha yüksek çıkmıştır fakat aralarında çok da büyük bir fark yoktur.
- Effect size ise area ile diğer iki değişken arasında yaklaşık 2.6 çıkması bu iki değişkenin de area'ya çok fazla etkisi olduğunu göstermektedir. Ayrıca compactness ve radius arasındaki cohen's d değeri yaklaşık 5.6 çıkmıştır. Bu ise bu ikilinin birbirine etkisinin çok daha büyük olduğunu göstermektedir. Eğer sadece korelasyonlara baksaydık ve effect size'a bakmasaydık compactness'ın diğer iki değişken üzerinde bu kadar büyük etkisi olduğunu kaçırmış olacaktık. Effect size sayesinde bu üç değişkenin üçünün de birbiri üzerinde çok etkili olduğunu gözlemlemiş olduk.
- Son olarak kovaryans da korelasyon sonucuna benzer bir şekilde sonuç verdi. Radius ve area arasında çok yüksek bir kovaryans çıktı. Diğer iki kovaryans değeri ise düşük çıkmamasına karşın radius ve area arasındaki kadar da yüksek çıkmadı. Korelasyon değerleri pozitif çıktığı için kovaryans değerleri de pozitif çıkmıştır ve bu da değişimin üç değişkende de aynı yönde olduğunu göstermektedir.

## Soru 4

#### Örnek 1:
Bir davranışın bir hastalığa etkisini hesaplamak için kullanılabilir. Mesela sigara içen bir kişinin kanseri olma ihtimalini hesaplamak istersek:<br>
$P(A) \rightarrow $ kanser olma olasılığı<br>
$P(B) \rightarrow$ sigara içme olasılığı<br>
$P(B|A) \rightarrow$ kanser olan birinin sigara içiyor olma olasılığı<br>
Bu 3 olasılık, sigara içen bir kişinin kanser olma olasılığına göre çok daha kolay bulunabilecek veya verilerden çok daha kolay hesaplanabilecek olasılıklar olduğu için, bu olasılıklar ve bayes teoremi kullanılarak sigara içen bir kişinin kanser olma ihtimali hesaplanabilir.<br><br>
Sigara içen bir kişinin kanser olma olasılığı$ \rightarrow P(A|B) = \frac{P(B|A)P(A)}{P(B)}$

***
#### Örnek 2:
Spam filtreleme için kullanılmaktadır. Bir kelimeyi içeren bir mailin spam olma olasılığını hesaplamak için kullanılabilir. Bunun en basit ve doğrudan yolu:<br>
$P(A) \rightarrow $ spam olma olasılığı<br>
$P(B) \rightarrow$ spesifik bir kelimenin bulunma olasılığı<br>
$P(B|A) \rightarrow$ spam olan bir mail'de o spesifik kelimenin bulunma olasılığı<br>
Yine aynı şekilde eldeki verilerden bu 3 değeri hesaplamak gayet kolaydır. Bayes teoremini de kullanarak sonuca ulaşılabilir:<br>
O kelimenin bulunduğu mail'in spam olma olasılığı: $ \rightarrow P(A|B) = \frac{P(B|A)P(A)}{P(B)} $


***
#### Örnek 3:
Covid-19 test sonucunun doğru olma ihtimalini hesaplamak için kullanılabilir:<br>
$P(A) \rightarrow $ hasta olma ihtimali<br>
$P(B) \rightarrow$ testin pozitif çıkma ihtimali<br>
$P(B|A) \rightarrow$ gerçekten hasta olduğunda testin pozitif çıkma ihtimali<br>
Test pozitif çıktığında gerçekten hasta olma ihtimali: $ \rightarrow P(A|B) = \frac{P(B|A)P(A)}{P(B)} $


***
#### Örnek 4:
Hava durumu tahmini için kullanılabilir. Örneğin gökyüzünde bulut olduğunda yağmur yağma ihtimali hesaplanabilir.<br>
$P(A) \rightarrow $ yağmur yağma ihtimali<br>
$P(B) \rightarrow$ gökyüzünde bulut olma ihtimali<br>
$P(B|A) \rightarrow$ yağmur yağdığında gökyüzünde bulut olma ihtimali(her zaman olacağı için 1'e eşittir)<br>
Gökyüzünde bulut olduğunda yağmur yağma ihtimali: $ \rightarrow P(A|B) = \frac{P(A)}{P(B)} $

***
#### Örnek 5:
Gaussian Naive Bayes, Multinomial Naive Bayes ve Bernoulli Naive Bayes; Bayes teoremini kullanarak oluşturulmuş algoritmalardır. Bir çok modelde sıklıkla kullanılan bu modeller sınıflandırma algoritmalarıdır. Bu algoritmanın temelinde Bayes teoreminin yanı sıra masum(naive) bir kabul kullanılmıştır. Bu 'masum' kabul bütün parametrelerin birbirinden istatistiksel olarak bağımsız olduğu ve birbirini etkilemediği kabuludür.

## Soru 5

-|Poisson| Normal | Gamma| Weilbull | Logaritmik 
:---:|:---:|:---:|:---:|:---:|:---:|
PDF:|-|$${f(x)}=\frac{1}{\sigma{\sqrt{2\pi}}} e^{-\frac{1}{2}\left(\frac{x-\mu }{\sigma}\right)^{2}} $$|$$ {f(x)=}\frac{1}{\Gamma (k)\theta^k}x^{k-1}e^{-\frac{x}{\theta}}$$|$${f(x)=}{\begin{cases}{\frac {k}{\lambda }}\left({\frac {x}{\lambda }}\right)^{k-1}e^{-(x/\lambda )^{k}} & x\geq 0\\0 & x<0\end{cases}}$$|-
PMF:|$$f(x)=\frac{\lambda^{k}e^{-\lambda }}{k!}$$|-|-|-|$$f(x)={\frac{-1}{\ln(1-p)}}{\frac {p^{k}}{k}}$$
Ayrık/Devamlı:|Ayrık|Devamlı|Devamlı|Devamlı|Ayrık
Özel Durumlar:|${\lambda}$=ortalama=varyans|$\mu$=ortalama=medyan=mod|iki formu var(k$\lambda$ ve $\alpha\beta$)|x<0 durumunda olasılık her zaman 0|mod =1
Kullanım Alanları:|Çok fazla etkisi bulunan ama gerçekleşme olasılığı düşük olaylarda kullanışlıdır|'Düzgün' dağılan ve günlük hayatta karşımıza çıkan bir çok alanda kullanılır|Bekleme sürelerinin dağılımı için sıklıkla kullanılır.|Az gerçekleşen ve ekstrem olaylar için sıklıkla kullanılır.|Belirli bir sürede gerçekleşen bir olayın sayısı gibi dağılımlarda kullanılabilir
Örnek:|Radyoaktif bir maddenin alfa parçacığı yayması|boy,kilo,IQ vb.(Günlük hayattan bir çok dağılım)|Bir ürünün ömrünü test ederken oluşan dağılım, kanser hastalarının yaş dağılımı|Başarısızlık analizi,ani yüksek voltaj oluşumları,güvenilirlik testleri|Belirli bir sürede bir müşterinin aldığı ürün sayısı, belirli bir sürede ışığın etrafında toplanan farklı böcek türlerinin sayısı
Notlar:|Binom dağılımda deneme sayısı sonsuza, gerçekleşme olasılığı 0'a yaklaşırsa Poisson dağılıma yakınsar.|- Ortalamadan sırayla 1,2 ve 3 sigma uzaklaşıldığında toplam dağılımın %68, %95 ve %99.7'si kapsanmış olur. - Bu dağılımlar içerisinde simetrik olan tek dağılımdır.|||Log-normal dağılımla aynı şey değildir.

Not: Devamlı dağılım fonksiyonlarının probability distribution function(PDF)'ı olur, ayrık dağılım fonksiyonlarının probability mass function(PMF)'ı olur.

## Grafikler(PDF/PMF):
<h3>Poisson:</h3><br>
<img src="https://upload.wikimedia.org/wikipedia/commons/1/16/Poisson_pmf.svg" width=500px style="background-color:white;"><br>
<h3>Normal:</h3><br>
<img src="https://upload.wikimedia.org/wikipedia/commons/7/74/Normal_Distribution_PDF.svg" width=500px style="background-color:white;"><br>
<h3>Gamma:</h3><br>
<img src="https://upload.wikimedia.org/wikipedia/commons/e/e6/Gamma_distribution_pdf.svg" width=500px style="background-color:white;"><br>
<h3>Weibull:</h3><br>
<img src="https://upload.wikimedia.org/wikipedia/commons/5/58/Weibull_PDF.svg" width=500px style="background-color:white;"><br>
<h3>Logaritmik:</h3><br>
<img src="https://upload.wikimedia.org/wikipedia/commons/2/20/Logarithmicpmf.svg" width=500px style="background-color:white;"><br>
</h3>

# Kaynaklar:
[Effect Size(Wikipedia)](https://en.wikipedia.org/wiki/Effect_size#Cohen's_d)<br>
[Poisson Distribution(Wikipedia)](https://en.wikipedia.org/wiki/Poisson_distribution)<br>
[Normal Distribution(Wikipedia)](https://en.wikipedia.org/wiki/Normal_distribution)<br>
[Gamma Distribution(Wikipedia)](https://en.wikipedia.org/wiki/Gamma_distribution)<br>
[Weibull Distribution(Wikipedia)](https://en.wikipedia.org/wiki/Weibull_distribution)<br>
[Logarithmic Distribution(Wikipedia)](https://en.wikipedia.org/wiki/Logarithmic_distribution)<br>